# Creating the web scraper

In [291]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

# Getting the dataset

In [61]:
data = pd.read_csv('charts.csv')

In [583]:
refined_data = data.loc[(data["rank"] == 1)]

In [327]:
new_index = [1,2,3,4,5,6,7,8,9,10,11,12,133,14,15,16,17,18,19,20]

In [584]:
new_data = refined_data.reset_index(drop = True)

In [585]:
refined_data.shape

(227253, 9)

In [586]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

Int64Index([     0,      4,      5,      6,      7,      8,      9,     10,
                11,     12,
            ...
            227242, 227243, 227244, 227245, 227246, 227247, 227249, 227250,
            227251, 227252],
           dtype='int64', length=193123)

In [588]:
new_data.drop(index_names, inplace=True)

In [589]:
new_data.shape

(34130, 9)

In [590]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0
25,Rockabye (feat. Sean Paul & Anne-Marie),1,2017-01-01,Clean Bandit,https://open.spotify.com/track/5knuzwU65gJK7IF...,Hungary,top200,MOVE_UP,6263.0
...,...,...,...,...,...,...,...,...,...
227143,Diamond in the rough,1,2021-07-30,Arik Ancelin,https://open.spotify.com/track/54AtiA08i3sDfJY...,Iceland,viral50,MOVE_UP,NaN
227144,Aadmi Chutiya Hai,1,2021-07-30,Rahgir,https://open.spotify.com/track/2PEJKORCdmceTW2...,India,viral50,MOVE_UP,NaN
227196,Bezos I,1,2021-07-31,Bo Burnham,https://open.spotify.com/track/0Kdj7nwaYQmvhxn...,Czech Republic,viral50,MOVE_UP,NaN
227202,Paar tundi veel,1,2021-07-31,Shanon,https://open.spotify.com/track/3LM9EDiw6CtKhCt...,Estonia,viral50,MOVE_UP,NaN


In [591]:
contain_values = new_data[new_data.artist.str.contains('Shakira')]
print (contain_values)

                          title  rank        date                    artist  \
98      Chantaje (feat. Maluma)     1  2017-01-02                   Shakira   
281     Chantaje (feat. Maluma)     1  2017-01-03                   Shakira   
583     Chantaje (feat. Maluma)     1  2017-01-04                   Shakira   
3133    Chantaje (feat. Maluma)     1  2017-01-07                   Shakira   
6260    Chantaje (feat. Maluma)     1  2017-01-12                   Shakira   
6779    Chantaje (feat. Maluma)     1  2017-01-14                   Shakira   
7657    Chantaje (feat. Maluma)     1  2017-01-15                   Shakira   
83302               Clandestino     1  2018-07-04           Shakira, Maluma   
83336               Clandestino     1  2018-07-04           Shakira, Maluma   
83714               Clandestino     1  2018-07-04           Shakira, Maluma   
84110               Clandestino     1  2018-07-05           Shakira, Maluma   
84111               Clandestino     1  2018-07-05   

In [411]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [330]:
new_data.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
4,Reggaetón Lento (Bailemos),1,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Bolivia,top200,SAME_POSITION,6784.0


In [331]:
new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [332]:
new_data.shape

(21, 9)

In [354]:
new_data.iloc[140087:140100,:]

,title,rank,date,artist,url,region,chart,trend,streams
140087,Falling,1,2019-12-05,Trevor Daniel,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Egypt,viral50,SAME_POSITION,NaN
140088,Tusa,1,2019-12-05,"KAROL G, Nicki Minaj",https://open.spotify.com/track/7k4t7uLgtOxPwTp...,El Salvador,viral50,SAME_POSITION,NaN
140089,Falling,1,2019-12-05,Trevor Daniel,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Estonia,viral50,SAME_POSITION,NaN
140090,Falling,1,2019-12-05,Trevor Daniel,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Finland,viral50,SAME_POSITION,NaN
140091,Bim Bam toi,1,2019-12-05,Carla,https://open.spotify.com/track/33NTeQz7sVYLmkJ...,France,viral50,MOVE_UP,NaN
140092,Falling,1,2019-12-05,Trevor Daniel,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Germany,viral50,MOVE_UP,NaN
140093,Lebih Dari Egoku,1,2019-12-05,Mawar de Jongh,https://open.spotify.com/track/4vh193RZbEEBanS...,Global,viral50,SAME_POSITION,NaN
140094,Falling,1,2019-12-05,Trevor Daniel,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Greece,viral50,SAME_POSITION,NaN
140095,Jenny (I Wanna Ruin Our Friendship),1,2020-12-07,Studio Killers,https://open.spotify.com/track/7lgA34nFXxjqQgy...,Iceland,viral50,MOVE_UP,NaN
140096,501,1,2021-03-14,"Bilal Wahib, Ronnie Flex",https://open.spotify.com/track/0Fl4PBfIwC5G1IQ...,Netherlands,top200,SAME_POSITION,304282.0


In [342]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
4,Reggaetón Lento (Bailemos),1,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Bolivia,top200,SAME_POSITION,6784.0
...,...,...,...,...,...,...,...,...,...
227248,double take,1,2021-07-31,dhruv,https://open.spotify.com/track/2CVChktEKBsM6v4...,United Arab Emirates,viral50,MOVE_UP,NaN
227249,Diamond in the rough,1,2021-07-31,Arik Ancelin,https://open.spotify.com/track/54AtiA08i3sDfJY...,United Kingdom,viral50,SAME_POSITION,NaN
227250,Diamond in the rough,1,2021-07-31,Arik Ancelin,https://open.spotify.com/track/54AtiA08i3sDfJY...,United States,viral50,SAME_POSITION,NaN
227251,Beggin',1,2021-07-31,Måneskin,https://open.spotify.com/track/3Wrjm47oTz2sjIg...,Uruguay,viral50,SAME_POSITION,NaN


# Extracting artist and track name

In [199]:
def get_track_name(nr):
    

    track_string = str(new_data.title[nr])
    
    x = track_string.find(" (")
    #y = artist_string.find("'")
    
    if(int(x) < 0):
        track_string = track_string
    else:
        track_string = track_string[:x]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(new_data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [592]:
get_track_name(98)

'Chantaje'

In [593]:
get_track_artist(98)

'Shakira'

# Getting URL

In [240]:
def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    x = track.find(" ")
    y = artist.find(" ")
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [594]:
get_url(98)

'https://genius.com/Shakira-Chantaje-lyrics'

# Scraping credits

In [262]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')


        featur = []
        writer = []
        label = []
        producer = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')


            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    featur.append(x)



            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)



            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":featur, "Writer":writer, "Label":label, "Producer":producer}        
        
        
    return collection

In [628]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
extLis = extract_credits(get_url(98))

# Dict to csv

In [629]:
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

In [630]:
df = pd.DataFrame.from_dict(credit_list) 
df.to_csv (r'C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\test8.csv', index = False, header=True)

In [631]:
df

,Feature,Writer,Label,Producer
0,Maluma,Kevin ADG,None,The Rudeboyz
1,None,Chan El Genio,None,Shakira
2,None,Kenai,None,Maluma
3,None,Maluma,None,None
4,None,Shakira,None,None


***

# Neo4j

In [451]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [452]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test123")


In [545]:
conn.query('''MATCH (n) RETURN n;''')

[]

In [632]:
artist = get_track_artist(98)
track = get_track_name(98)
track

'Chantaje'

In [633]:
artist

'Shakira'

### Different querys depending on the information in the csv file

In [634]:
# 200403
conn.query('''LOAD CSV WITH HEADERS FROM "file:///test8.csv" AS row
WITH row WHERE row.Feature IS NOT NULL
MERGE (a:Person {name:row.Feature, title:"Artist"})
MERGE (a1:Person {name:"''' + artist + '''", title:"Artist"})
MERGE (l:Label {name:row.Label})
CREATE (s:Song {name:"''' + track + '''"})
MERGE (a)-[:COLLABORATED]-(a1)
MERGE (a)-[:FEATURED]->(s)<-[:SANG]-(a1)
MERGE (l)-[:RELEASED]->(s)
''')

Query failed! {code: Neo.ClientError.Statement.SemanticError} {message: Cannot merge the following node because of null property value for 'name': (:Label {name: null})}


In [620]:
# WITHOUT FEATURE
# 218900
conn.query('''LOAD CSV WITH HEADERS FROM "file:///test8.csv" AS row

WITH row WHERE row.Label IS NOT NULL
MERGE (l:Label {name:row.Label})

MERGE (p:Person {name:"''' + artist + '''", title:"Artist"})

MERGE (s:Song {name:"''' + track + '''"})

MERGE (p)-[:SANG]->(s)
MERGE (l)-[:RELEASED]->(s)
''')

[]

In [635]:
conn.query('''LOAD CSV WITH HEADERS FROM "file:///test8.csv" AS row
WITH row WHERE row.Feature IS NOT NULL
MERGE (a:Person {name:row.Feature, title:"Artist"})
MERGE (a1:Person {name:"''' + artist + '''", title:"Artist"})
CREATE (s:Song {name:"''' + track + '''"})
MERGE (a)-[:COLLABORATED]-(a1)
MERGE (a)-[:FEATURED]->(s)<-[:SANG]-(a1)
''')

[]

In [536]:
df

,Feature,Writer,Label,Producer
0,None,The Weeknd,XO Records,Oscar Holter
1,None,Belly,Republic Records,Max Martin
2,None,DaHeala,None,The Weeknd
3,None,Oscar Holter,None,None
4,None,Max Martin,None,None
